In [1]:
import numpy as np
import pandas as pd

## 三、 商户数据解读与探索

对官方的商户数据与信用卡交易数据进行分析和清洗。

historical_transaction是某个时间点以前的信用卡消费记录；

new_merchant_transaction是某个时间点之后的

merchangts是商户的属性

主办方给的拓展信息：信用卡交易记录和商户相关数据。同时包含了训练集、、测试集中所有信用卡的部分记录

- 规律一致性分析的实际应用

可得出许多可用于后续指导建模的意见。

基本结论：
1. 如果分布非常一致，则说明取自同一整体，训练集和测试集一致性较高，模型效果上限较高，建模时应更加依靠特征工程和模型建模技巧
2. 如果不太一致，模型预测效果受限，且容易过拟合，实际建模时多考虑交叉验证等防止过拟合，和trick的使用

一般数据解读、探索、初步清洗时同步的。

解读：快速获得基本信息，通过比对官方的字段解释，快速了解字段基本含义

探索：快速了解数据集的基本数据情况，主要包括
- 数据正确性校验，
    - id 是否一致；
    - 字段排布和数据字典所列举的字段是否一致
    - 字段类型是否合规
        - python不支持直接代入object类，要转成int或plot
- 数据质量分析。
快速了解各字段基本情况：数据类型、是否数据不一致、重复值、缺失值等。

清洗：
- 在建模/特征工程前进行必要的调整，以确保后续操作可执行，包括字段类型调整、重复值处理、缺失值处理等。
- 清理时可以先简单的缺失值填补，后续建模时可根据实际建模结果来调整填补策略。

<img src="./data preprocessing.png">

## 1. 数据解读
相对较小的商户信息表

In [2]:
merchant = pd.read_csv('../data/competitions/elo-merchant-category-recommendation/merchants.csv', header=0)

In [3]:
merchant.head(5)

,merchant_id,merchant_group_id,merchant_category_id,subsector_id,numerical_1,numerical_2,category_1,most_recent_sales_range,most_recent_purchases_range,avg_sales_lag3,...,avg_sales_lag6,avg_purchases_lag6,active_months_lag6,avg_sales_lag12,avg_purchases_lag12,active_months_lag12,category_4,city_id,state_id,category_2
0,M_ID_838061e48c,8353,792,9,-0.057471,-0.057471,N,E,E,-0.40,...,-2.25,18.666667,6,-2.32,13.916667,12,N,242,9,1.0
1,M_ID_9339d880ad,3184,840,20,-0.057471,-0.057471,N,E,E,-0.72,...,-0.74,1.291667,6,-0.57,1.687500,12,N,22,16,1.0
2,M_ID_e726bbae1e,447,690,1,-0.057471,-0.057471,N,E,E,-82.13,...,-82.13,260.000000,2,-82.13,260.000000,2,N,-1,5,5.0
3,M_ID_a70e9c5f81,5026,792,9,-0.057471,-0.057471,Y,E,E,NaN,...,NaN,4.666667,6,NaN,3.833333,12,Y,-1,-1,NaN
4,M_ID_64456c37ce,2228,222,21,-0.057471,-0.057471,Y,E,E,NaN,...,NaN,0.361111,6,NaN,0.347222,12,Y,-1,-1,NaN


In [4]:
merchant.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 334696 entries, 0 to 334695
Data columns (total 22 columns):
 #   Column                       Non-Null Count   Dtype  
---  ------                       --------------   -----  
 0   merchant_id                  334696 non-null  object 
 1   merchant_group_id            334696 non-null  int64  
 2   merchant_category_id         334696 non-null  int64  
 3   subsector_id                 334696 non-null  int64  
 4   numerical_1                  334696 non-null  float64
 5   numerical_2                  334696 non-null  float64
 6   category_1                   334696 non-null  object 
 7   most_recent_sales_range      334696 non-null  object 
 8   most_recent_purchases_range  334696 non-null  object 
 9   avg_sales_lag3               334683 non-null  float64
 10  avg_purchases_lag3           334696 non-null  float64
 11  active_months_lag3           334696 non-null  int64  
 12  avg_sales_lag6               334683 non-null  float64
 13 

基本字段解释

In [5]:
df = pd.read_excel('../data/competitions/elo-merchant-category-recommendation/Data_Dictionary.xlsx', header=2,
                   sheet_name='merchant')
df

,Columns,Description
0,merchant_id,Unique merchant identifier
1,merchant_group_id,Merchant group (anonymized )
2,merchant_category_id,Unique identifier for merchant category (anony...
3,subsector_id,Merchant category group (anonymized )
4,numerical_1,anonymized measure
5,numerical_2,anonymized measure
6,category_1,anonymized category
7,most_recent_sales_range,Range of revenue (monetary units) in last acti...
8,most_recent_purchases_range,Range of quantity of transactions in last acti...
9,avg_sales_lag3,Monthly average of revenue in last 3 months di...


| 1                           | 2                                                                                              |
|-----------------------------|------------------------------------------------------------------------------------------------|
| merchant_id                 | Unique merchant identifier                                                                     |
| merchant_group_id           | Merchant group (anonymized )                                                                   |
| merchant_category_id        | Unique identifier for merchant category (anonymized )                                          |
| subsector_id                | Merchant category group (anonymized )                                                          |
| numerical_1                 | anonymized measure                                                                             |
| numerical_2                 | anonymized measure                                                                             |
| category_1                  | anonymized category                                                                            |
| most_recent_sales_range     | Range of revenue (monetary units) in last active month --> A > B > C > D > E                   |
| most_recent_purchases_range | Range of quantity of transactions in last active month --> A > B > C > D > E                   |
| avg_sales_lag3              | Monthly average of revenue in last 3 months divided by revenue in last active month            |
| avg_purchases_lag3          | Monthly average of transactions in last 3 months divided by transactions in last active month  |
| active_months_lag3          | Quantity of active months within last 3 months                                                 |
| avg_sales_lag6              | Monthly average of revenue in last 6 months divided by revenue in last active month            |
| avg_purchases_lag6          | Monthly average of transactions in last 6 months divided by transactions in last active month  |
| active_months_lag6          | Quantity of active months within last 6 months                                                 |
| avg_sales_lag12             | Monthly average of revenue in last 12 months divided by revenue in last active month           |
| avg_purchases_lag12         | Monthly average of transactions in last 12 months divided by transactions in last active month |
| active_months_lag12         | Quantity of active months within last 12 months                                                |
| category_4                  | anonymized category                                                                            |
| city_id                     | City identifier (anonymized )                                                                  |
| state_id                    | State identifier (anonymized )                                                                 |
| category_2                  | anonymized category                                                                            |


most rescent sales range, 没有告诉什么叫活跃，可以认为是营收状况；是离散、有序的

一般，机器学习不区分离散型变量的细分。实际上，有两个类别：
- 有序型：数字有顺序的含义
- 名义变量：如，男=1，女=0，这时没有1>0

有时会把有序变量看成连续变量

提供了商户属性（类别，商品种类等），同时也提供了商户近期的交易数据

avg_purchases_lag：月平均交易量除以上个活跃月份

近期与长期的对比

## 2. 数据探索

- 正确性校验
商户id出现次数

In [6]:
print(merchant.shape, merchant['merchant_id'].nunique())

(334696, 22) 334633


不是一个id对应一条数据，存在一个商户有多条记录。

由于商户特征较多，可验证商户数据特征是否和数据字典中特征一致：

In [7]:
print(pd.Series(merchant.columns.tolist()).sort_values().values == pd.Series(df['Columns'].tolist()).sort_values().values)

[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True]


商户特征一致

- 缺失值分析

In [8]:
merchant.isnull().sum()

merchant_id                        0
merchant_group_id                  0
merchant_category_id               0
subsector_id                       0
numerical_1                        0
numerical_2                        0
category_1                         0
most_recent_sales_range            0
most_recent_purchases_range        0
avg_sales_lag3                    13
avg_purchases_lag3                 0
active_months_lag3                 0
avg_sales_lag6                    13
avg_purchases_lag6                 0
active_months_lag6                 0
avg_sales_lag12                   13
avg_purchases_lag12                0
active_months_lag12                0
category_4                         0
city_id                            0
state_id                           0
category_2                     11887
dtype: int64

缺失列为avg_lag，是近期与长期的比。

有可能这13个是新商户，且可能相同的13家商户，都不到3个月

另一个为category_2，是匿名字段

## 3. 数据预处理

- 离散/连续字段标注
    - 商户数据集同时存在分类变量和离散变量，因此可以首先根据字段的说明对不同属性特征进行统一划分：

In [9]:
category_cols = ['merchant_id', 'merchant_group_id', 'merchant_category_id', 'subsector_id', 'category_1', 'most_recent_sales_range', 'most_recent_purchases_range', 'category_4', 'city_id', 'state_id', 'category_2'
]
numeric_cols = ['numerical_1', 'numerical_2', 'avg_sales_lag3', 'avg_purchases_lag3', 'active_months_lag3', 'avg_sales_lag6', 'avg_purchases_lag6', 'active_months_lag6', 'avg_sales_lag12', 'avg_purchases_lag12', 'active_months_lag12']

# 检验特征是否划分完全
assert len(category_cols) + len(numeric_cols) == merchant.shape[1]

可能有情况是：有些变量可以看作连续型，也可以是离散型

在预处理时，可以先统一划分成离散型，到建模阶段可以再调整

- 离散变量数据情况
    - 简单查看

In [10]:
# 查看取值水平
merchant[category_cols].nunique()

merchant_id                    334633
merchant_group_id              109391
merchant_category_id              324
subsector_id                       41
category_1                          2
most_recent_sales_range             5
most_recent_purchases_range         5
category_4                          2
city_id                           271
state_id                           25
category_2                          5
dtype: int64

匿名字段category_1，category_2，取值水平不同

In [11]:
# 查看分类变量目前的类型
merchant[category_cols].dtypes

merchant_id                     object
merchant_group_id                int64
merchant_category_id             int64
subsector_id                     int64
category_1                      object
most_recent_sales_range         object
most_recent_purchases_range     object
category_4                      object
city_id                          int64
state_id                         int64
category_2                     float64
dtype: object

未来要对object类进行处理

In [12]:
# 查看离散变量的缺失值情况
merchant[category_cols].isnull().sum()

merchant_id                        0
merchant_group_id                  0
merchant_category_id               0
subsector_id                       0
category_1                         0
most_recent_sales_range            0
most_recent_purchases_range        0
category_4                         0
city_id                            0
state_id                           0
category_2                     11887
dtype: int64

- 离散变量的缺失值标注
    - category_2存在较多缺失值，由于该分类取值水平为1-5，因此，可先将缺失值标记为-1：

In [13]:
merchant['category_2'].unique()

array([ 1.,  5., nan,  2.,  3.,  4.])

In [14]:
merchant['category_2'] = merchant['category_2'].fillna(-1)

接下来处理object类

python没有算法可以处理

如果是名义变量可以直接one-hot编码

- 离散变量字典编码

将object类对象按照sort顺序进行数值化（整数）编码。

例如原始category_1取值为Y/N，通过sort排序后N在Y前，因此重新编码时为0，Y为1。

注意点：
- 编码过程会对原始的对象类型先进行排序
    - 当对开卡月份排序时，会存在有些月份没有的情况，比如2、5、7、10，排序时就会变成1、2、3、4，与月份数字的含义不同，背景信息缺失。特别是处理时序数据时，而且需要更复杂的方法

严格来说，变量类型有3类：连续型、名义型、有序型。

名义变量没有大小的意义

In [15]:
# 字典编码函数
def change_object_cols(se):
    value = se.unique().tolist()
    value.sort()
    return se.map(pd.Series(range(len(value)), index=value)).values

测试函数效果

In [16]:
merchant['category_1']

0         N
1         N
2         N
3         Y
4         Y
         ..
334691    N
334692    Y
334693    N
334694    Y
334695    N
Name: category_1, Length: 334696, dtype: object

In [17]:
change_object_cols(merchant['category_1'])

array([0, 0, 0, ..., 0, 1, 0])

对merchant对象中的四个object类型列进行类别转化：

In [18]:
for col in ['category_1', 'most_recent_sales_range', 'most_recent_purchases_range', 'category_4']:
    change_object_cols(merchant[col])

现在这些列都是int类型，可以代入进行建模

- 连续变量的数据探索

In [19]:
# 查看连续变量的类型
merchant[numeric_cols].dtypes

numerical_1            float64
numerical_2            float64
avg_sales_lag3         float64
avg_purchases_lag3     float64
active_months_lag3       int64
avg_sales_lag6         float64
avg_purchases_lag6     float64
active_months_lag6       int64
avg_sales_lag12        float64
avg_purchases_lag12    float64
active_months_lag12      int64
dtype: object

In [20]:
# 连续变量的缺失情况
merchant[numeric_cols].isnull().sum()

numerical_1             0
numerical_2             0
avg_sales_lag3         13
avg_purchases_lag3      0
active_months_lag3      0
avg_sales_lag6         13
avg_purchases_lag6      0
active_months_lag6      0
avg_sales_lag12        13
avg_purchases_lag12     0
active_months_lag12     0
dtype: int64

可能是某13个商户

考虑到merchant数据集有30多万条，13条缺失不算什么，因此可以使用均值填充，没有必要使用更高级的方法，因为影响不大。

In [21]:
# 连续变量整体情况
merchant[numeric_cols].describe()

E:\py_project\StudyTorch\.venv\Lib\site-packages\pandas\core\nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)
E:\py_project\StudyTorch\.venv\Lib\site-packages\pandas\core\nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)
E:\py_project\StudyTorch\.venv\Lib\site-packages\pandas\core\nanops.py:1016: RuntimeWarning: invalid value encountered in subtract
  sqr = _ensure_numeric((avg - values) ** 2)


,numerical_1,numerical_2,avg_sales_lag3,avg_purchases_lag3,active_months_lag3,avg_sales_lag6,avg_purchases_lag6,active_months_lag6,avg_sales_lag12,avg_purchases_lag12,active_months_lag12
count,334696.000000,334696.000000,334683.000000,3.346960e+05,334696.000000,3.346830e+05,3.346960e+05,334696.000000,3.346830e+05,3.346960e+05,334696.000000
mean,0.011476,0.008103,13.832993,inf,2.994108,2.165079e+01,inf,5.947397,2.522771e+01,inf,11.599335
std,1.098154,1.070497,2395.489999,NaN,0.095247,3.947108e+03,NaN,0.394936,5.251842e+03,NaN,1.520138
min,-0.057471,-0.057471,-82.130000,3.334953e-01,1.000000,-8.213000e+01,1.670447e-01,1.000000,-8.213000e+01,9.832954e-02,1.000000
25%,-0.057471,-0.057471,0.880000,9.236499e-01,3.000000,8.500000e-01,9.022475e-01,6.000000,8.500000e-01,8.983333e-01,12.000000
50%,-0.057471,-0.057471,1.000000,1.016667e+00,3.000000,1.010000e+00,1.026961e+00,6.000000,1.020000e+00,1.043361e+00,12.000000
75%,-0.047556,-0.047556,1.160000,1.146522e+00,3.000000,1.230000e+00,1.215575e+00,6.000000,1.290000e+00,1.266480e+00,12.000000
max,183.735111,182.079322,851844.640000,inf,3.000000,1.513959e+06,inf,6.000000,2.567408e+06,inf,12.000000


由于连续型变量存在部分缺失值，且部分连续变量还存在无穷值inf，需要进行简单处理。

inf也是不合规的数据，不能用来建模

- 无穷值处理
    - 采用天花板盖帽法，将inf改为该列的最大显式数值。

In [22]:
import numpy as np

In [23]:
inf_cols = ['avg_purchases_lag3', 'avg_purchases_lag6', 'avg_purchases_lag12']
merchant[inf_cols] = merchant[inf_cols].replace(np.inf, merchant[inf_cols].replace(np.inf, -99).max().max())

In [24]:
merchant[numeric_cols].describe()

,numerical_1,numerical_2,avg_sales_lag3,avg_purchases_lag3,active_months_lag3,avg_sales_lag6,avg_purchases_lag6,active_months_lag6,avg_sales_lag12,avg_purchases_lag12,active_months_lag12
count,334696.000000,334696.000000,334683.000000,334696.000000,334696.000000,3.346830e+05,334696.000000,334696.000000,3.346830e+05,334696.000000,334696.000000
mean,0.011476,0.008103,13.832993,2.145143,2.994108,2.165079e+01,2.441947,5.947397,2.522771e+01,2.633572,11.599335
std,1.098154,1.070497,2395.489999,213.955844,0.095247,3.947108e+03,209.439373,0.394936,5.251842e+03,205.206198,1.520138
min,-0.057471,-0.057471,-82.130000,0.333495,1.000000,-8.213000e+01,0.167045,1.000000,-8.213000e+01,0.098330,1.000000
25%,-0.057471,-0.057471,0.880000,0.923650,3.000000,8.500000e-01,0.902247,6.000000,8.500000e-01,0.898333,12.000000
50%,-0.057471,-0.057471,1.000000,1.016667,3.000000,1.010000e+00,1.026961,6.000000,1.020000e+00,1.043361,12.000000
75%,-0.047556,-0.047556,1.160000,1.146522,3.000000,1.230000e+00,1.215575,6.000000,1.290000e+00,1.266480,12.000000
max,183.735111,182.079322,851844.640000,61851.333333,3.000000,1.513959e+06,61851.333333,6.000000,2.567408e+06,61851.333333,12.000000


- 缺失值处理
    - 缺失值处理方法有很多。该数据集缺失数据较少，先简单采用均值进行填补处理，后续根据需要进行优化。

In [25]:
for col in numeric_cols:
    merchant[col] = merchant[col].fillna(merchant[col].mean())

In [26]:
merchant[numeric_cols].describe()

,numerical_1,numerical_2,avg_sales_lag3,avg_purchases_lag3,active_months_lag3,avg_sales_lag6,avg_purchases_lag6,active_months_lag6,avg_sales_lag12,avg_purchases_lag12,active_months_lag12
count,334696.000000,334696.000000,334696.000000,334696.000000,334696.000000,3.346960e+05,334696.000000,334696.000000,3.346960e+05,334696.000000,334696.000000
mean,0.011476,0.008103,13.832993,2.145143,2.994108,2.165079e+01,2.441947,5.947397,2.522771e+01,2.633572,11.599335
std,1.098154,1.070497,2395.443476,213.955844,0.095247,3.947031e+03,209.439373,0.394936,5.251740e+03,205.206198,1.520138
min,-0.057471,-0.057471,-82.130000,0.333495,1.000000,-8.213000e+01,0.167045,1.000000,-8.213000e+01,0.098330,1.000000
25%,-0.057471,-0.057471,0.880000,0.923650,3.000000,8.500000e-01,0.902247,6.000000,8.500000e-01,0.898333,12.000000
50%,-0.057471,-0.057471,1.000000,1.016667,3.000000,1.010000e+00,1.026961,6.000000,1.020000e+00,1.043361,12.000000
75%,-0.047556,-0.047556,1.160000,1.146522,3.000000,1.230000e+00,1.215575,6.000000,1.290000e+00,1.266480,12.000000
max,183.735111,182.079322,851844.640000,61851.333333,3.000000,1.513959e+06,61851.333333,6.000000,2.567408e+06,61851.333333,12.000000


至此完成了商户数据的预处理。

# 二、信用卡交易数据解读与探索

交易数据是最大的数据集

## 1.数据解读与验证

先解释数据集，简单验证数据集正确性。

信用卡交易记录：historical_transactions和new_merchant_transactions。

字段类似，记录不同时间区间

- historical_transactions
    - 记录信用卡在特定商户、3个月内的消费记录。约有2.6G，并非必要建模字段，但能提取有效信息。

In [27]:
history_transaction = pd.read_csv('../data/competitions/elo-merchant-category-recommendation/historical_transactions.csv', header=0)

In [28]:
history_transaction.head(5)

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id
0,Y,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_e020e9b302,-8,-0.703331,2017-06-25 15:33:07,1.0,16,37
1,Y,C_ID_4e6213e9bc,88,N,0,A,367,M_ID_86ec983688,-7,-0.733128,2017-07-15 12:10:45,1.0,16,16
2,Y,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_979ed661fc,-6,-0.720386,2017-08-09 22:04:29,1.0,16,37
3,Y,C_ID_4e6213e9bc,88,N,0,A,560,M_ID_e6d5ae8ea6,-5,-0.735352,2017-09-02 10:06:26,1.0,16,34
4,Y,C_ID_4e6213e9bc,88,N,0,A,80,M_ID_e020e9b302,-11,-0.722865,2017-03-10 01:14:19,1.0,16,37


In [29]:
history_transaction.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29112361 entries, 0 to 29112360
Data columns (total 14 columns):
 #   Column                Dtype  
---  ------                -----  
 0   authorized_flag       object 
 1   card_id               object 
 2   city_id               int64  
 3   category_1            object 
 4   installments          int64  
 5   category_3            object 
 6   merchant_category_id  int64  
 7   merchant_id           object 
 8   month_lag             int64  
 9   purchase_amount       float64
 10  purchase_date         object 
 11  category_2            float64
 12  state_id              int64  
 13  subsector_id          int64  
dtypes: float64(2), int64(6), object(6)
memory usage: 3.0+ GB


将近3千万条数据，14个字段

In [30]:
pd.read_excel('../data/competitions/elo-merchant-category-recommendation/Data Dictionary.xlsx', header=2, sheet_name='history')

,Columns,Description
0,card_id,Card identifier
1,month_lag,month lag to reference date
2,purchase_date,Purchase date
3,authorized_flag,"Y' if approved, 'N' if denied"
4,category_3,anonymized category
5,installments,number of installments of purchase
6,category_1,anonymized category
7,merchant_category_id,Merchant category identifier (anonymized )
8,subsector_id,Merchant category group identifier (anonymized )
9,merchant_id,Merchant identifier (anonymized)


authorized_flag：官方没解释授权什么。

是否公开；是否授权交易；特殊商家等。忽略背景进行建模

merchant_category_id 是商户组别的id

- new_merchant_tansactions: 近期交易信息

2018年2月后的交易信息。

字段一致

In [31]:
new_transaction = pd.read_csv('../data/competitions/elo-merchant-category-recommendation/new_merchant_transactions.csv', header=0)

In [32]:
new_transaction.head(5)

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id
0,Y,C_ID_415bb3a509,107,N,1,B,307,M_ID_b0c793002c,1,-0.557574,2018-03-11 14:57:36,1.0,9,19
1,Y,C_ID_415bb3a509,140,N,1,B,307,M_ID_88920c89e8,1,-0.569580,2018-03-19 18:53:37,1.0,9,19
2,Y,C_ID_415bb3a509,330,N,1,B,507,M_ID_ad5237ef6b,2,-0.551037,2018-04-26 14:08:44,1.0,9,14
3,Y,C_ID_415bb3a509,-1,Y,1,B,661,M_ID_9e84cda3b1,1,-0.671925,2018-03-07 09:43:21,NaN,-1,8
4,Y,C_ID_ef55cf8d4b,-1,Y,1,B,166,M_ID_3c86fa3831,1,-0.659904,2018-03-22 21:07:53,NaN,-1,29


In [33]:
new_transaction.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1963031 entries, 0 to 1963030
Data columns (total 14 columns):
 #   Column                Dtype  
---  ------                -----  
 0   authorized_flag       object 
 1   card_id               object 
 2   city_id               int64  
 3   category_1            object 
 4   installments          int64  
 5   category_3            object 
 6   merchant_category_id  int64  
 7   merchant_id           object 
 8   month_lag             int64  
 9   purchase_amount       float64
 10  purchase_date         object 
 11  category_2            float64
 12  state_id              int64  
 13  subsector_id          int64  
dtypes: float64(2), int64(6), object(6)
memory usage: 209.7+ MB


- 对比merchant数据集

先查看有哪些字段一致

In [34]:
duplicate_cols = []

for col in merchant.columns:
    if col in new_transaction.columns:
        duplicate_cols.append(col)

print(duplicate_cols)

['merchant_id', 'merchant_category_id', 'subsector_id', 'category_1', 'city_id', 'state_id', 'category_2']


交易记录中的merchant_id不唯一：

In [35]:
# 取出和商户数据表重复字段并去重
new_transaction[duplicate_cols].drop_duplicates().shape

(291242, 7)

后续拼接表时，主键只能有1个，card_id。

In [36]:
# 商户id去重
new_transaction['merchant_id'].nunique()

226129

原因可能是商铺在经营过程中动态变化。

因此，后续建模时，优先使用交易记录表的相应记录。

## 2. 数据预处理

- 连续/离散字段标注

该数据集存在一个时间列，将其单独归为一类：

也可以看成离散型，但如果想从时序关系中提取更多信息，需要单独划为一列

In [37]:
category_cols = ['authorized_flag', 'card_id', 'city_id', 'category_1', 'category_3', 'merchant_category_id', 'merchant_id', 'category_2', 'subsector_id', 'state_id']
numeric_cols = ['installments', 'month_alg', 'purchase_amount']
time_cols =['purchase_date']

assert len(numeric_cols) + len(category_cols) + len(time_cols) == new_transaction.shape[1]

- 字段类型转化/缺失值填补

In [38]:
# 查看分类变量的类别
new_transaction[category_cols].dtypes

authorized_flag          object
card_id                  object
city_id                   int64
category_1               object
category_3               object
merchant_category_id      int64
merchant_id              object
category_2              float64
subsector_id              int64
state_id                  int64
dtype: object

In [39]:
new_transaction[category_cols].isnull().sum()

authorized_flag              0
card_id                      0
city_id                      0
category_1                   0
category_3               55922
merchant_category_id         0
merchant_id              26216
category_2              111745
subsector_id                 0
state_id                     0
dtype: int64

对object对象进行字典编码（id除外）

缺失值可以用众数填补（因为缺失的不多），这里用-1填补：

In [40]:
for col in ['category_3', 'authorized_flag', 'category_1']:
    new_transaction[col] = change_object_cols(new_transaction[col].fillna(-1).astype(str))

new_transaction[category_cols] = new_transaction[category_cols].fillna(-1)

同时进行字典编码和对缺失值进行标记（不是填补为-1）

In [41]:
new_transaction[category_cols].dtypes

authorized_flag           int64
card_id                  object
city_id                   int64
category_1                int64
category_3                int64
merchant_category_id      int64
merchant_id              object
category_2              float64
subsector_id              int64
state_id                  int64
dtype: object

# 三、数据清洗后数据生成

## 1. 回顾商户数据、交易数据清洗流程
简单总结：

### 商户数据 merchants.csv
- 划分连续字段和离散字段
- 对字符型离散字段进行字典排序编码
- 对缺失值处理，统一使用 -1 进行缺失值补充，本质上是一种标注
- 对连续型字段的无穷值进行处理，用该列的最大值进行替换
- 去除重复数据

### 交易数据 new_merchant_transactions.csv 和 historical_transactions.csv
- 划分字段类型，分为离散字段、连续字段、时间字段
- 和商户数据的处理方法一样，对字符型离散字段进行字典排序，对缺失值进行统一填充
- 对新生成的购买欲分离散字段进行字段排序编码
- 最后对多表进行拼接，并且通过 month_lag 字段是否大于0来进行区分

## 2. 创建清洗后数据
结合训练集和测试集的清洗流程，在此统一执行所有数据的数据清洗工作，并将其保存为本地文件，方便后续特征工程及算法建模过程使用